<a href="https://colab.research.google.com/github/niikun/ai_engineering_day3/blob/main/DualRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DualRAG RAG + Deep検索
https://arxiv.org/abs/2504.18243   
https://zenn.dev/knowledgesense/articles/10b2b5f772b810

RAGのソース文書に、正解が「そのまま」あればそれでOKですが、現実では普通、情報が分散しています。  
通常のRAGだと、ユーザーの質問に対して1回しか検索しないので、回答精度が悪いです。  
【ユーザーが質問を入力して来たとき】  

1.推論・検索（RaQ）
- 「現在の知識で十分か？」をメタ認知→「情報不足」と判断すると検索  
- 的確なキーワードで検索できるよう、無駄な単語は省く（=エンティティの抽出）
2.情報の要約（pKA）
- 1で取得した情報を整理・要約
3.1・2の繰り返し
- 回答にたどり着くまで、必要なだけこのサイクルを繰り返す
4.最終回答を生成  

DualRAGという手法のキモは、「検索⇄要約」のサイクルです。単に検索回数を多くするのではなく、常に「メタ的な視点から要約・整理するステップ」を挟みます。そうすることで、最低限の検索回数で、答えにたどり着くことが可能です。

In [1]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_openai langchain-community langchain-chroma

In [4]:
question = "LLMにおけるInference Time Scalingとは？"
gold_answer = "「Inference Time Scaling」とは、推論時に計算量を増やしてモデルの性能を高める手法です。これはモデルのサイズを大きくする代わりに、難しい入力に対して多くの計算リソースを使うことで、より良い出力を得ようとするアプローチです。"

In [2]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52 (from 1)
Receiving objects: 100% (52/52), 83.21 KiB | 5.55 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [8]:
import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["LANGSMITH_API_KEY"] = userdata.get('lang_smith')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
%env LANGCHAIN_TRACING_V2=true
%env LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
%env LANGSMITH_PROJECT="DualRAG"

model = ChatOpenAI(model_name="gpt-4o-mini")

# テキストローダーの準備
loader = TextLoader("/content/lecture-ai-engineering/day3/data/LLM2024_day4.txt")
docs = loader.load()

# テキストのチャンク分け
text_splitter = CharacterTextSplitter(
    chunk_size=1000,chunk_overlap=200
)
all_splits = text_splitter.split_documents(docs)

# チャンクの埋め込み準備
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

db = Chroma.from_documents(all_splits,embeddings)
retriever = db.as_retriever()

env: LANGCHAIN_TRACING_V2=true
env: LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
env: LANGSMITH_PROJECT="DualRAG"


In [13]:
from typing import List, Dict, Any
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import BaseRetriever, Document

retriever: BaseRetriever = retriever

# プロンプト定義
meta_prompt = ChatPromptTemplate.from_template(
    "ユーザーの質問: {question}\n"
    "現時点の文脈:\n{context}\n"
    "→ 上記の情報だけで回答できますか？"
)

extract_prompt = ChatPromptTemplate.from_template(
    "質問: {question}\n"
    "→ 検索に使うキーワード（エンティティ）を列挙してください。"
)

summarize_prompt = ChatPromptTemplate.from_template(
    "検索結果:\n{snippets}\n"
    "→ 上記を要点だけ短く要約してください。"
)

final_prompt = ChatPromptTemplate.from_template(
    "最終コンテキスト:\n{context}\n"
    "質問: {question}\n"
    "→ 回答を生成してください。"
)

def iterative_rag(question: str, max_cycles: int = 3) -> str:
    context = ""  # 初期文脈は空、もしくは事前知識
    for cycle in range(max_cycles):
        # 1) メタ認知フェーズ（RaQ）
        msg = meta_prompt.format_messages(context=context, question=question)
        decision = llm(msg).content
        if "回答可能" in decision:
            break

        # 2) キーワード抽出
        msg = extract_prompt.format_messages(question=question)
        keywords = llm(msg).content

        # 3) 検索フェーズ
        docs: List[Document] = retriever.get_relevant_documents(keywords)
        snippets = "\n".join(d.page_content for d in docs[:5])

        # 4) 要約フェーズ（pKA）
        msg = summarize_prompt.format_messages(snippets=snippets)
        summary = llm(msg).content

        # 5) コンテキストに追加
        context += "\n" + summary

    # 6) 最終回答フェーズ
    msg = final_prompt.format_messages(context=context, question=question)
    return llm(msg).content

# 使い方
answer = iterative_rag(question)
print(answer)


LLM（大規模言語モデル）におけるInference Time Scaling（推論時スケーリング）は、モデルが学習時にパラメータ数やデータ量を増やすのではなく、推論時に使用する計算資源を増加させるアプローチを指します。この概念は、推論時の計算能力を高めることで、モデルの応答速度や精度を向上させることを目指しています。

具体的には、推論時の計算資源を増やすことによって、以下のようなメリットが得られる可能性があります：

1. **応答の精度向上**: 簡単な質問には迅速に答えを得ることができ、複雑な問題に対してはより多くの計算資源を投入することで、深い考察やより高精度の回答が可能になる。

2. **柔軟な応答生成**: モデルが状況に応じて計算資源を動的に調整できることで、条件に応じた最適な応答が実現できる。

3. **リソースの効率化**: 特定のタスクや文脈に合わせて計算資源を調整することによって、高いパフォーマンスを維持しながらコストを管理することができる。

最近の研究、特にGoogle DeepMindの発表などは、同じ計算資源条件下でパラメータを増やすよりも、推論時の資源を増やすことがより効果的である可能性があることを示唆しています。ただし、この選択が最良であるかどうかはタスク依存であるため、使用する状況や目的に応じた適切な判断が求められます。

全体として、Inference Time Scalingは、LLMの性能を向上させるための新しいアプローチとして注目されており、モデルの能力を最大限に引き出す鍵となる要素とされています。


In [14]:
from openai import OpenAI
from google.colab import userdata
OPEN_AI_API_KEY = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPEN_AI_API_KEY, max_retries=5, timeout=60)

def openai_generator(query):
    messages = [
        {
            "role":"user",
            "content":query
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content

def evaluate_answer_accuracy(query, response, reference):

    template_accuracy1 = (
          "Instruction: You are a world class state of the art assistant for rating "
          "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
          "Say 4, if User Answer is full contained and equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
          "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
          "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
          "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
          "### Question: {query}\n"
          "### {answer0}: {sentence_inference}\n"
          "### {answer1}: {sentence_true}\n"
          "The rating is:\n"
      )
    template_accuracy2 = (
          "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
          "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
          "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
          "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
          "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
          "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
          "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
          "Question: {query}\n\n"
          "{answer0}: {sentence_inference}\n\n"
          "{answer1}: {sentence_true}\n\n"
          "Rating: "
      )

    score1 = openai_generator(
                template_accuracy1.format(
                      query=query,
                      answer0="User Answer",
                      answer1="Reference Answer",
                      sentence_inference=response,
                      sentence_true=reference,
                    )
                )
    try:
      score1 = int(score1)
    except:
      print("Failed")
      score1 = 0

    score2 = openai_generator(
                template_accuracy2.format(
                        query=query,
                        answer0="Reference Answer",
                        answer1="User Answer",
                        sentence_inference=reference,
                        sentence_true=response,
                    )
                  )

    try:
      score2 = int(score2)
    except:
      print("Failed")
      score2 = 0


    return (score1 + score2) / 2

In [15]:
evaluate_answer_accuracy(question, answer, gold_answer)

2.0